In [1]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from sentence_transformers import SentenceTransformer
sentences = ["Isto é um exemplo", "Isto é um outro exemplo"]

model = SentenceTransformer('rufimelo/Legal-BERTimbau-sts-large-ma-v3')
embeddings = model.encode(sentences)
print(embeddings)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


[[-0.30714837 -0.8936157  -0.5609019  ... -0.01724652 -0.50345933
   0.41357243]
 [ 0.38083306 -0.51222104 -0.28849292 ... -0.3317719   0.07660595
   0.4618158 ]]


In [3]:
embeddings.shape

(2, 1024)

In [4]:
import json

with open("codigo_civil.json", "r") as f:
    cod_civil = json.load(f)

In [5]:
def collect_artigo_and_header(data):
    result = {'articles': [], 'contexts': []}

    def traverse(data):
        if isinstance(data, dict):
            for key, value in data.items():
                if key == 'artigos' and value:
                    result['articles'].append(value)
                if key == 'header' and value:
                    result['contexts'].append(value)
                # Recursive call to handle nested dictionaries
                if isinstance(value, dict):
                    traverse(value)
                elif isinstance(value, list):
                    traverse(value)
        elif isinstance(data, list):
            for item in data:
                # Recursive call to handle lists of dictionaries
                traverse(item)

    traverse(data)
    return result

result = collect_artigo_and_header(cod_civil)
all_context = result['contexts']
all_articles = result['articles']

In [6]:
len(all_context)

396

In [7]:
len(all_articles)

318

In [8]:
import pandas as pd

In [9]:
lengths = [len(article) for article in all_articles]
pd.Series(lengths).describe()

count    318.000000
mean       6.468553
std        4.647575
min        1.000000
25%        3.000000
50%        5.000000
75%        9.000000
max       24.000000
dtype: float64

In [10]:
lengths = [len('\n'.join(article)) for article in all_articles]
pd.Series(lengths).idxmax()

232

In [11]:
from tqdm import tqdm

embeddings_object = []
for article_group in tqdm(all_articles):
    embeddings = model.encode(article_group)
    mean_embedding = embeddings.mean(axis=0)
    embeddings_object.append({
        'embeddings': embeddings,
        'mean_embedding': mean_embedding
    })

import pickle
with open('codigocivil_embeddings_object.pkl', 'wb') as f:
    pickle.dump(embeddings_object, f)

100%|██████████| 318/318 [01:10<00:00,  4.51it/s]


In [12]:
with open('codigocivil_embeddings_object.pkl', 'rb') as f:
    embeddings_object = pickle.load(f)